 <center><h1>How I make a AI model to predict soccer game<h1></center>


![Drag Racing](./images/Soccer.png)

Here in Canada, it is the beginning of winter, going out becomes more and more difficult. People start to stay longer at home, they shorten their time spent outside. With this, it is the beginning of the world cup. 

And my bank account is empty. 💸

So I need a way to earn money without leaving my house. 

I quickly made the connection: I'm going to bet all my savings on the outcome of the World Cup.  But without taking any risk, in order to be sure of the result of the matches.

So I will try to build the perfect AI model to predict the results of the matches. 

## Step 1: Get the data

After long hours of research, I found this [website](https://www.football-data.co.uk/data.php) where I can find result of all the matchs from the 5 biggest European championship.

But this dataset alone will not be of much use to me. 

Among the most important parameters to create a coherent model, I need more information about the teams...

And for that, I thought that one of the most reliable sources would be the dataset of the game FIFA, which lists all the players of the big european championships, but also rates them, and gives me a lot of information that I can qualify as reliable and objective, even if they come from a video game

I found this dataset [here](https://www.kaggle.com/datasets/bryanb/fifa-player-stats-database?select=FIFA23_official_data.csv)



## Step 2: Clean the data

In [1]:
# This cell allow myself to have multiple outputs in one only cell 
# 
# https://www.roelpeters.be/jupyter-notebooks-how-to-print-multiple-outputs-in-a-cell/
from IPython.core.interactiveshell import InteractiveShell
import warnings
import json


In [2]:
# importantion de modules

import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import *
from sklearn.compose import *
import os
import glob
import difflib
from datetime import timedelta, date



In [3]:
pd.options.display.max_columns = 4000
pd.options.display.max_rows = 400
pd.options.mode.chained_assignment = None
InteractiveShell.ast_node_interactivity = "all"


In [4]:
# Constant
directory_PL_file = "./data/PremierLeague"
years_analysed = range(2017, 2023)


In [5]:
# merging the files
joined_files = os.path.join(directory_PL_file, "*.csv")

# A list of all joined files is returned
joined_list = glob.glob(joined_files)

# Finally, the files are joined
match = pd.concat(map(pd.read_csv, joined_list), ignore_index=True)

match.head()


,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,Bb1X2,BbMxH,BbAvH,BbMxD,BbAvD,BbMxA,BbAvA,BbOU,BbMx>2.5,BbAv>2.5,BbMx<2.5,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA,Time,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,E0,11/08/2017,Arsenal,Leicester,4,3,H,2,2,D,M Dean,27,6,10,3,9,12,9,4,0,1,0,0,1.53,4.5,6.50,1.50,4.60,6.75,1.47,4.5,6.50,1.44,4.40,6.50,1.53,4.55,6.85,1.53,4.2,6.00,1.53,4.50,6.50,41.0,1.55,1.51,4.60,4.43,6.89,6.44,37.0,1.65,1.61,2.43,2.32,21.0,-1.00,1.91,1.85,2.10,2.02,1.49,4.73,7.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,E0,12/08/2017,Brighton,Man City,0,2,A,0,0,D,M Oliver,6,14,2,4,6,9,3,10,0,2,0,0,11.00,5.5,1.33,11.00,5.25,1.30,8.00,5.3,1.35,10.00,5.00,1.30,10.95,5.55,1.34,10.00,4.8,1.33,10.00,5.50,1.33,40.0,11.50,10.10,5.60,5.25,1.36,1.32,35.0,1.70,1.63,2.40,2.27,20.0,1.50,1.95,1.91,2.01,1.96,11.75,6.15,1.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,E0,12/08/2017,Chelsea,Burnley,2,3,A,0,3,A,C Pawson,19,10,6,5,16,11,8,5,3,3,2,0,1.25,6.5,15.00,1.22,6.50,12.50,1.22,6.2,13.50,1.25,5.75,15.00,1.26,6.30,15.25,1.25,5.5,13.00,1.25,6.25,15.00,41.0,1.27,1.24,6.55,6.06,15.50,13.67,36.0,1.71,1.66,2.33,2.23,20.0,-1.75,2.03,1.97,1.95,1.90,1.33,5.40,12.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,E0,12/08/2017,Crystal Palace,Huddersfield,0,3,A,0,2,A,J Moss,14,8,4,6,7,19,12,9,1,3,0,0,1.83,3.6,5.00,1.80,3.50,4.75,1.85,3.5,4.30,1.80,3.40,4.60,1.83,3.58,5.11,1.80,3.3,5.00,1.83,3.60,5.00,41.0,1.86,1.81,3.65,3.50,5.11,4.82,36.0,2.19,2.11,1.79,1.72,18.0,-0.75,2.10,2.05,1.86,1.83,1.79,3.56,5.51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,E0,12/08/2017,Everton,Stoke,1,0,H,1,0,H,N Swarbrick,9,9,4,1,13,10,6,7,1,1,0,0,1.70,3.8,5.75,1.70,3.60,5.50,1.70,3.7,5.00,1.67,3.60,5.25,1.70,3.83,5.81,1.70,3.5,5.50,1.70,3.80,5.75,40.0,1.71,1.69,3.85,3.69,6.00,5.50,35.0,2.17,2.08,1.80,1.76,19.0,-0.75,1.94,1.90,2.01,1.98,1.82,3.49,5.42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
all_club_in_PL = match["HomeTeam"].unique()


In order to get all the club in Fifa, we need to get it from one dataset. We suppose that there is no club creation between years. In fact to be more precise, we assume that all the club we listed in the premier League are present in Fifa 22.

If a club is not in Premier league in Fifa 22 (but in the second division), there is no impact as we are just trying to make a connection beween dataset: we want to create a python dict that convert club from dataset match into club from fifa dataset

In [7]:
# Open Any Fifa dataset and get all the club into it
all_club_in_FIFA = pd.read_csv("./data/Fifa/FIFA22_official_data.csv")["Club"].astype(str).unique()


In [8]:
# Create the Dict
commonName = {}

# For each club in the dataset match, we try to see the most closest club string in the dataset
# from Fifa using difflib.get_close_matches 
# Source :
# https://stackoverflow.com/questions/10018679/python-find-closest-string-from-a-list-to-another-string

for club in all_club_in_PL:
    commonName[club] = difflib.get_close_matches(club, all_club_in_FIFA)

commonName

{'Arsenal': ['Arsenal', 'Arsenal Tula', 'Cesena'],
 'Brighton': [],
 'Chelsea': ['Chelsea', 'Cesena'],
 'Crystal Palace': ['Crystal Palace'],
 'Everton': ['Everton'],
 'Southampton': ['Southampton', 'Northampton Town'],
 'Watford': ['Watford', 'Waterford FC', 'Brentford'],
 'West Brom': ['West Bromwich Albion'],
 'Man United': ['Atlanta United', 'Manchester United', 'Sutton United'],
 'Newcastle': ['Newcastle Jets', 'Newcastle United'],
 'Bournemouth': ['AFC Bournemouth', 'Portsmouth'],
 'Burnley': ['Burnley', 'Barnsley', 'Bury'],
 'Leicester': ['Leicester City'],
 'Liverpool': ['Liverpool', 'Livorno', 'Liverpool Fútbol Club'],
 'Stoke': ['Stoke City'],
 'Swansea': ['Swansea City', 'Santos'],
 'Huddersfield': ['Huddersfield Town'],
 'Tottenham': ['Tottenham Hotspur'],
 'Man City': ['Swansea City', 'Manchester City', 'Mumbai City FC'],
 'West Ham': ['West Ham United'],
 'Fulham': ['Fulham'],
 'Wolves': [],
 'Cardiff': ['Cardiff City', 'Carpi'],
 'Aston Villa': ['Aston Villa'],
 'Norwich

As we see the result is not perfect: it cannot find any club for wolves, and sometimes, there is multiple choice. So I decided to go manually for the next step: create the dictionnary.
I just copy the results from the line above, and find what is corresponding in Fifa dataset.
What we have done cannot be entirely automatised, but it helps us a lot in order to make the dict quicker.

In [9]:
# Write the dict by hand, helped by the above code
PL_club = {
    "Arsenal": "Arsenal",
    "Brighton": "Brighton & Hove Albion",
    "Chelsea": "Chelsea",
    "Crystal Palace": "Crystal Palace",
    "Everton": "Everton",
    "Southampton": "Southampton",
    "Watford": "Watford",
    "West Brom": "West Bromwich Albion",
    "Man United": "Manchester United",
    "Newcastle": "Newcastle United",
    "Bournemouth": "AFC Bournemouth",
    "Burnley": "Burnley",
    "Leicester": "Leicester City",
    "Liverpool": "Liverpool",
    "Stoke": "Stoke City",
    "Swansea": "Swansea City",
    "Huddersfield": "Huddersfield Town",
    "Tottenham": "Tottenham Hotspur",
    "Man City": "Manchester City",
    "West Ham": "West Ham United",
    "Fulham": "Fulham",
    "Leeds": "Leeds United",
    "Aston Villa": "Aston Villa",
    "Wolves": "Wolverhampton Wanderers",
    "Brentford": "Brentford",
    "Nott'm Forest": "Nottingham Forest",
    "Cardiff": "Cardiff City",
    "Norwich": "Norwich City",
    "Sheffield United": "Sheffield United",
    "Hull": "Hull City",
    "Middlesbrough": "Middlesbrough",
    "Sunderland": "Sunderland",
}


In [10]:
# Save in file if we need to get the data later, or in another project
with open("./generated/club_pl.json", mode="w+") as outfile:
    json.dump(PL_club, outfile)


In [11]:
match.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,Bb1X2,BbMxH,BbAvH,BbMxD,BbAvD,BbMxA,BbAvA,BbOU,BbMx>2.5,BbAv>2.5,BbMx<2.5,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA,Time,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,E0,11/08/2017,Arsenal,Leicester,4,3,H,2,2,D,M Dean,27,6,10,3,9,12,9,4,0,1,0,0,1.53,4.5,6.50,1.50,4.60,6.75,1.47,4.5,6.50,1.44,4.40,6.50,1.53,4.55,6.85,1.53,4.2,6.00,1.53,4.50,6.50,41.0,1.55,1.51,4.60,4.43,6.89,6.44,37.0,1.65,1.61,2.43,2.32,21.0,-1.00,1.91,1.85,2.10,2.02,1.49,4.73,7.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,E0,12/08/2017,Brighton,Man City,0,2,A,0,0,D,M Oliver,6,14,2,4,6,9,3,10,0,2,0,0,11.00,5.5,1.33,11.00,5.25,1.30,8.00,5.3,1.35,10.00,5.00,1.30,10.95,5.55,1.34,10.00,4.8,1.33,10.00,5.50,1.33,40.0,11.50,10.10,5.60,5.25,1.36,1.32,35.0,1.70,1.63,2.40,2.27,20.0,1.50,1.95,1.91,2.01,1.96,11.75,6.15,1.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,E0,12/08/2017,Chelsea,Burnley,2,3,A,0,3,A,C Pawson,19,10,6,5,16,11,8,5,3,3,2,0,1.25,6.5,15.00,1.22,6.50,12.50,1.22,6.2,13.50,1.25,5.75,15.00,1.26,6.30,15.25,1.25,5.5,13.00,1.25,6.25,15.00,41.0,1.27,1.24,6.55,6.06,15.50,13.67,36.0,1.71,1.66,2.33,2.23,20.0,-1.75,2.03,1.97,1.95,1.90,1.33,5.40,12.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,E0,12/08/2017,Crystal Palace,Huddersfield,0,3,A,0,2,A,J Moss,14,8,4,6,7,19,12,9,1,3,0,0,1.83,3.6,5.00,1.80,3.50,4.75,1.85,3.5,4.30,1.80,3.40,4.60,1.83,3.58,5.11,1.80,3.3,5.00,1.83,3.60,5.00,41.0,1.86,1.81,3.65,3.50,5.11,4.82,36.0,2.19,2.11,1.79,1.72,18.0,-0.75,2.10,2.05,1.86,1.83,1.79,3.56,5.51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,E0,12/08/2017,Everton,Stoke,1,0,H,1,0,H,N Swarbrick,9,9,4,1,13,10,6,7,1,1,0,0,1.70,3.8,5.75,1.70,3.60,5.50,1.70,3.7,5.00,1.67,3.60,5.25,1.70,3.83,5.81,1.70,3.5,5.50,1.70,3.80,5.75,40.0,1.71,1.69,3.85,3.69,6.00,5.50,35.0,2.17,2.08,1.80,1.76,19.0,-0.75,1.94,1.90,2.01,1.98,1.82,3.49,5.42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
match = match[["Date","HomeTeam","AwayTeam","FTHG","FTAG"]]
match= match.replace({"HomeTeam": PL_club,"AwayTeam":PL_club })
match.head()
match.info()

,Date,HomeTeam,AwayTeam,FTHG,FTAG
0,11/08/2017,Arsenal,Leicester City,4,3
1,12/08/2017,Brighton & Hove Albion,Manchester City,0,2
2,12/08/2017,Chelsea,Burnley,2,3
3,12/08/2017,Crystal Palace,Huddersfield Town,0,3
4,12/08/2017,Everton,Stoke City,1,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2280 entries, 0 to 2279
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Date      2280 non-null   object
 1   HomeTeam  2280 non-null   object
 2   AwayTeam  2280 non-null   object
 3   FTHG      2280 non-null   int64 
 4   FTAG      2280 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 89.2+ KB


In [13]:
match["Date"] = pd.to_datetime(match["Date"], dayfirst=True)
match.head()
match.info()


for year in years_analysed:
    start_dt = pd.to_datetime(date(year-1, 8, 1))
    end_dt = pd.to_datetime(date(year, 8, 1))
    mask = (match["Date"] >= start_dt) & (match["Date"] < end_dt)
    match.loc[mask, "Season"] = year
    if match.loc[match["Season"] == year].shape[0] != 380:
        print(
            "Only "
            + str(match.loc[match["Season"] == year].shape[0])
            + " match found for "
            + str(year)
            + " Season"
        )

match["Season"] = match["Season"].astype(int)
match.head()


,Date,HomeTeam,AwayTeam,FTHG,FTAG
0,2017-08-11,Arsenal,Leicester City,4,3
1,2017-08-12,Brighton & Hove Albion,Manchester City,0,2
2,2017-08-12,Chelsea,Burnley,2,3
3,2017-08-12,Crystal Palace,Huddersfield Town,0,3
4,2017-08-12,Everton,Stoke City,1,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2280 entries, 0 to 2279
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      2280 non-null   datetime64[ns]
 1   HomeTeam  2280 non-null   object        
 2   AwayTeam  2280 non-null   object        
 3   FTHG      2280 non-null   int64         
 4   FTAG      2280 non-null   int64         
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 89.2+ KB


,Date,HomeTeam,AwayTeam,FTHG,FTAG,Season
0,2017-08-11,Arsenal,Leicester City,4,3,2018
1,2017-08-12,Brighton & Hove Albion,Manchester City,0,2,2018
2,2017-08-12,Chelsea,Burnley,2,3,2018
3,2017-08-12,Crystal Palace,Huddersfield Town,0,3,2018
4,2017-08-12,Everton,Stoke City,1,0,2018


In [14]:
all_club_in_FIFA = pd.read_csv("./data/Fifa/FIFA22_official_data.csv")["Club"].astype(str).unique()
fifa_data = {} 

for year in years_analysed:
    fifa_data[year] = pd.read_csv("./data/Fifa/FIFA{}_official_data.csv".format(abs(year) % 100 ))
    fifa_data[year].head(2)

,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Best Position,Best Overall Rating
0,176580,L. Suárez,29,https://cdn.sofifa.com/players/176/580/17_60.png,Uruguay,https://cdn.sofifa.com/flags/uy.png,92,92,FC Barcelona,https://cdn.sofifa.com/teams/241/light_30.png,€83M,€525K,2291,Right,5.0,4.0,4.0,High/ Medium,Normal (170-185),Yes,"<span class=""pos pos25"">ST",9.0,"Jul 11, 2014",NaN,2021,6'0,190lbs,77.0,94.0,77.0,83.0,88.0,86.0,86.0,84.0,64.0,91.0,88.0,77.0,86.0,93.0,60.0,87.0,69.0,89.0,80.0,86.0,78.0,41.0,92.0,84.0,85.0,83.0,30.0,45.0,38.0,27.0,25.0,31.0,33.0,37.0,ST,88.0
1,178518,R. Nainggolan,28,https://cdn.sofifa.com/players/178/518/17_60.png,Belgium,https://cdn.sofifa.com/flags/be.png,86,86,Roma,https://cdn.sofifa.com/teams/52/light_30.png,€37.5M,€130K,2290,Right,3.0,3.0,3.0,High/ High,Stocky (170-185),No,"<span class=""pos pos22"">LF",4.0,"Jan 7, 2014",NaN,2021,5'9,143lbs,73.0,76.0,59.0,84.0,75.0,80.0,73.0,68.0,81.0,85.0,81.0,78.0,81.0,87.0,84.0,84.0,76.0,94.0,78.0,86.0,88.0,86.0,88.0,76.0,63.0,85.0,78.0,85.0,88.0,11.0,11.0,14.0,8.0,11.0,CDM,84.0


,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Best Position,Best Overall Rating,Release Clause
0,176580,L. Suárez,30,https://cdn.sofifa.com/players/176/580/18_60.png,Uruguay,https://cdn.sofifa.com/flags/uy.png,92,92,FC Barcelona,https://cdn.sofifa.com/teams/241/light_30.png,€97M,€500K,2321,Right,5.0,4.0,4.0,High/ Medium,Normal (170-185),Yes,"<span class=""pos pos26"">LS",9.0,"Jul 11, 2014",NaN,2021,6'0,190lbs,77.0,94.0,77.0,83.0,88.0,86.0,86.0,84.0,64.0,91.0,88.0,77.0,86.0,93.0,78.0,87.0,69.0,89.0,83.0,86.0,87.0,41.0,92.0,84.0,85.0,85.0,30.0,45.0,38.0,27.0,25.0,31.0,33.0,37.0,CF,88.0,€198.9M
1,178518,R. Nainggolan,29,https://cdn.sofifa.com/players/178/518/18_60.png,Belgium,https://cdn.sofifa.com/flags/be.png,86,86,Roma,https://cdn.sofifa.com/teams/52/light_30.png,€42.5M,€130K,2286,Right,3.0,3.0,3.0,High/ High,Stocky (170-185),Yes,"<span class=""pos pos13"">RCM",4.0,"Jan 7, 2014",NaN,2021,5'9,170lbs,73.0,76.0,59.0,84.0,75.0,80.0,73.0,68.0,81.0,85.0,80.0,76.0,81.0,87.0,84.0,84.0,76.0,94.0,76.0,86.0,88.0,86.0,88.0,76.0,63.0,84.0,78.0,86.0,88.0,11.0,11.0,14.0,8.0,11.0,CDM,84.0,€72.3M


,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Best Position,Best Overall Rating,Release Clause
0,176580,L. Suárez,31,https://cdn.sofifa.com/players/176/580/19_60.png,Uruguay,https://cdn.sofifa.com/flags/uy.png,91,91,FC Barcelona,https://cdn.sofifa.com/teams/241/light_30.png,€80M,€450K,2349,Right,5.0,4.0,3.0,High/ Medium,Normal (170-185),Yes,"<span class=""pos pos25"">ST",9.0,"Jul 11, 2014",NaN,2021,6'0,190lbs,77.0,92.0,82.0,82.0,90.0,86.0,86.0,84.0,69.0,88.0,83.0,73.0,82.0,92.0,83.0,86.0,69.0,90.0,83.0,85.0,87.0,41.0,92.0,84.0,85.0,85.0,62.0,45.0,38.0,27.0,25.0,31.0,33.0,37.0,ST,87.0,€164M
1,194765,A. Griezmann,27,https://cdn.sofifa.com/players/194/765/19_60.png,France,https://cdn.sofifa.com/flags/fr.png,89,89,Atlético Madrid,https://cdn.sofifa.com/teams/240/light_30.png,€71M,€140K,2293,Left,4.0,3.0,4.0,High/ High,Lean (170-185),Yes,"<span class=""pos pos24"">RS",7.0,"Jul 28, 2014",NaN,2023,5'9,161lbs,82.0,89.0,84.0,84.0,87.0,87.0,86.0,84.0,78.0,90.0,88.0,85.0,90.0,90.0,82.0,82.0,90.0,86.0,62.0,82.0,73.0,49.0,91.0,85.0,83.0,89.0,59.0,54.0,48.0,14.0,8.0,14.0,13.0,14.0,LW,87.0,€145.6M


,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Best Position,Best Overall Rating,Release Clause,DefensiveAwareness
0,176580,L. Suárez,32,https://cdn.sofifa.com/players/176/580/20_60.png,Uruguay,https://cdn.sofifa.com/flags/uy.png,89,89,FC Barcelona,https://cdn.sofifa.com/teams/241/light_30.png,€53M,€350K,2327,Right,5.0,4.0,3.0,High/ Medium,Normal (170-185),Yes,"<span class=""pos pos25"">ST",9.0,"Jul 11, 2014",NaN,2021,6'0,190lbs,80.0,91.0,83.0,82.0,90.0,85.0,86.0,82.0,77.0,84.0,76.0,70.0,79.0,92.0,79.0,88.0,69.0,82.0,86.0,86.0,87.0,41.0,92.0,84.0,83.0,85.0,NaN,45.0,38.0,27.0,25.0,31.0,33.0,37.0,ST,89.0,€108.7M,57.0
1,194765,A. Griezmann,28,https://cdn.sofifa.com/players/194/765/20_60.png,France,https://cdn.sofifa.com/flags/fr.png,89,89,FC Barcelona,https://cdn.sofifa.com/teams/241/light_30.png,€69M,€370K,2299,Left,4.0,3.0,4.0,High/ High,Normal (170-185),Yes,"<span class=""pos pos27"">LW",17.0,"Jul 12, 2019",NaN,2024,5'9,161lbs,83.0,89.0,84.0,85.0,87.0,88.0,86.0,85.0,82.0,90.0,82.0,79.0,90.0,92.0,83.0,82.0,90.0,87.0,63.0,83.0,73.0,49.0,90.0,86.0,86.0,89.0,NaN,54.0,49.0,14.0,8.0,14.0,13.0,14.0,CF,89.0,€141.5M,59.0


,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Best Position,Best Overall Rating,Release Clause,DefensiveAwareness
0,176580,L. Suárez,33,https://cdn.sofifa.com/players/176/580/20_60.png,Uruguay,https://cdn.sofifa.com/flags/uy.png,87,87,Atlético Madrid,https://cdn.sofifa.com/teams/240/light_30.png,€31.5M,€115K,2316,Right,5.0,4.0,3.0,High/ Medium,PLAYER_BODY_TYPE_374,Yes,"<span class=""pos pos24"">RS",9.0,"Sep 25, 2020",NaN,2022,6'0,190lbs,80.0,92.0,84.0,83.0,90.0,84.0,86.0,82.0,77.0,84.0,72.0,68.0,76.0,92.0,78.0,89.0,69.0,78.0,85.0,88.0,87.0,41.0,91.0,84.0,83.0,85.0,NaN,45.0,38.0,27.0,25.0,31.0,33.0,37.0,ST,87.0,€64.6M,57.0
1,192985,K. De Bruyne,29,https://cdn.sofifa.com/players/192/985/20_60.png,Belgium,https://cdn.sofifa.com/flags/be.png,91,91,Manchester City,https://cdn.sofifa.com/teams/10/light_30.png,€87M,€370K,2304,Right,4.0,5.0,4.0,High/ High,PLAYER_BODY_TYPE_321,Yes,"<span class=""pos pos13"">RCM",17.0,"Aug 30, 2015",NaN,2023,5'11,154lbs,94.0,82.0,55.0,94.0,82.0,88.0,85.0,83.0,93.0,92.0,77.0,76.0,78.0,91.0,76.0,91.0,63.0,89.0,74.0,91.0,76.0,66.0,88.0,94.0,84.0,91.0,NaN,65.0,53.0,15.0,13.0,5.0,10.0,13.0,CAM,91.0,€161M,68.0


,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Preferred Foot,International Reputation,Weak Foot,Skill Moves,Work Rate,Body Type,Real Face,Position,Jersey Number,Joined,Loaned From,Contract Valid Until,Height,Weight,Crossing,Finishing,HeadingAccuracy,ShortPassing,Volleys,Dribbling,Curve,FKAccuracy,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Reactions,Balance,ShotPower,Jumping,Stamina,Strength,LongShots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Best Position,Best Overall Rating,Release Clause,DefensiveAwareness
0,212198,Bruno Fernandes,26,https://cdn.sofifa.com/players/212/198/22_60.png,Portugal,https://cdn.sofifa.com/flags/pt.png,88,89,Manchester United,https://cdn.sofifa.com/teams/11/30.png,€107.5M,€250K,2341,Right,3.0,3.0,4.0,High/ High,Unique,Yes,"<span class=""pos pos18"">CAM",18.0,"Jan 30, 2020",NaN,2025,179cm,69kg,87.0,83.0,64.0,91.0,87.0,83.0,87.0,87.0,88.0,87.0,77.0,73.0,80.0,91.0,79.0,89.0,73.0,91.0,70.0,89.0,78.0,66.0,87.0,90.0,91.0,87.0,NaN,73.0,65.0,12.0,14.0,15.0,8.0,14.0,CAM,88.0,€206.9M,72.0
1,209658,L. Goretzka,26,https://cdn.sofifa.com/players/209/658/22_60.png,Germany,https://cdn.sofifa.com/flags/de.png,87,88,FC Bayern München,https://cdn.sofifa.com/teams/21/30.png,€93M,€140K,2314,Right,4.0,4.0,3.0,High/ Medium,Unique,Yes,"<span class=""pos pos11"">LDM",8.0,"Jul 1, 2018",NaN,2026,189cm,82kg,75.0,82.0,86.0,86.0,69.0,84.0,76.0,75.0,84.0,87.0,78.0,83.0,76.0,88.0,71.0,85.0,79.0,88.0,88.0,86.0,81.0,86.0,85.0,84.0,60.0,82.0,NaN,85.0,77.0,13.0,8.0,15.0,11.0,9.0,CM,87.0,€160.4M,74.0


In [15]:
for year in years_analysed:
    selected_player = fifa_data[year].loc[fifa_data[year]['ID'] == 176580]
    print("in {} {} has {} years".format(year,selected_player["Name"].iloc[0].strip(),selected_player["Age"].iloc[0]))


in 2017 L. Suárez has 29 years
in 2018 L. Suárez has 30 years
in 2019 L. Suárez has 31 years
in 2020 L. Suárez has 32 years
in 2021 L. Suárez has 33 years
in 2022 L. Suárez has 34 years


![Drag Racing](./images/positionsmap.jpg)

In [16]:
data_positionning = {
    "DEF": ["LWB", "LB", "CB", "RB", "RWB"],
    "GOAL": ["GK"],
    "ATT": ["LW", "LF", "ST", "CF", "RF", "RW"],
    "MID": ["LM", "CM", "CAM", "CDM", "RM"],
    "LEFT": ["LB", "LWB", "LM", "LW", "LF"],
    "RIGHT": ["RB", "RWB", "RM", "RW", "RF"],
    "CENTER": ["CB", "CDM", "CM", "CAM", "ST", "CF"],
}



In [17]:
row = [
    "Overall",
    "Potential",
    "Value",
    "Wage",
    "International Reputation",
    "Weak Foot",
    "Skill Moves",
    "Crossing",
    "Finishing",
    "HeadingAccuracy",
    "ShortPassing",
    "Volleys",
    "Dribbling",
    "Curve",
    "FKAccuracy",
    "LongPassing",
    "BallControl",
    "Acceleration",
    "SprintSpeed",
    "Agility",
    "Reactions",
    "Balance",
    "ShotPower",
    "Jumping",
    "Stamina",
    "Strength",
    "LongShots",
    "Aggression",
    "Interceptions",
    "Positioning",
    "Vision",
    "Penalties",
    "Composure",
    "Marking",
    "StandingTackle",
    "SlidingTackle",
    "GKDiving",
    "GKHandling",
    "GKKicking",
    "GKPositioning",
    "GKReflexes",
    "Best Overall Rating",
]
for year in years_analysed:
    for PL_club_value in PL_club:
        data = fifa_data[year].loc[
            fifa_data[year]["Club"] == PL_club[PL_club_value]
        ]
        

        data["Value"] = data["Value"].str.replace("M", "000000")
        data["Value"] = data["Value"].str.replace("K", "000")
        data["Value"] = data["Value"].str.replace(".", "", regex=True)
        data["Value"] = data["Value"].str.replace("€", "", regex=True)
        data["Value"] = data["Value"].astype(int)

        data["Wage"] = data["Wage"].str.replace("M", "000000")
        data["Wage"] = data["Wage"].str.replace("K", "000")
        data["Wage"] = data["Wage"].str.replace(".", "", regex=True)
        data["Wage"] = data["Wage"].str.replace("€", "", regex=True)
        data["Wage"] = data["Wage"].astype(int)

        for pos in data_positionning:
            
            data_pos = data.loc[
                data["Best Position"].isin(data_positionning[pos])
            ]
            data_pos = data_pos[row]

            mask = (match["HomeTeam"] == PL_club[PL_club_value])
            mean = data_pos.mean()  
            elements = ['HOME {0} {1}'.format(pos, element) for element in mean.keys().tolist()]
            match.loc[mask, elements] = mean.tolist()

            mask = (match["AwayTeam"] == PL_club[PL_club_value])
            mean = data_pos.mean()  
            elements = ['AWAY {0} {1}'.format(pos, element) for element in mean.keys().tolist()]
            match.loc[mask, elements] = mean.tolist()


In [19]:
from pathlib import Path  
filepath = Path('./out.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
match.to_csv(filepath, index=False)  